**Import library**

In [35]:
from openai import AzureOpenAI
import pyodbc
import pandas as pd

import os
from dotenv import load_dotenv
from pathlib import Path

import re
from datetime import datetime

import sys
sys.path.append(str(Path(os.getcwd()).parent / "service"))
from database import DatabaseService

**Locate env path**

In [36]:
script_dir = Path(os.getcwd()).parent
env_path = script_dir / '.env'
load_dotenv(env_path)

True

**Object Oriented Programming**

In [37]:
class AzureAI:

    def __init__(self):

        self.sql_file_path = script_dir / "sql_script" / "Schema.sql"

        self.model_name = os.getenv("openai_model_name")

        self.api_key = os.getenv("openai_model_api_key")
        self.api_version = os.getenv("openai_api_version")
        self.azure_endpoint = os.getenv("openai_api_endpoint")

        self.database = os.getenv('azure_database')
        self.schema = os.getenv('azure_schema')
    
    def ai_response(self, user_prompt, create_sql_script=False):

        with open(self.sql_file_path, 'r', encoding='utf-8') as file:
            schema_query = file.read()

        conn = DatabaseService().create_connection()

        schema_df = pd.read_sql_query(schema_query, conn)
        schema_df = schema_df[schema_df['TABLE_CATALOG'] == f'{self.database}']
        schema_df = schema_df[schema_df['TABLE_SCHEMA'] == f'{self.schema}']

        schema_text = ""
        for (schema, table), group in schema_df.groupby(['TABLE_SCHEMA', 'TABLE_NAME']):
            columns = ", ".join(f"{row.COLUMN_NAME} ({row.DATA_TYPE})" for _, row in group.iterrows())
            schema_text += f"Table {schema}.{table}: {columns}\n"

        user_prompt = user_prompt

        model_name = self.model_name

        client = AzureOpenAI(
            api_key=self.api_key,
            api_version=self.api_version,
            azure_endpoint=self.azure_endpoint,
        )

        system_message = f"""
        You are a helpful assistant that writes SQL Server queries.
        Here is the database schema:

        {schema_text}
        """

        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": f"Write a SQL Server query for this: {user_prompt}"}
            ]
        )

        generated_sql, result = response.choices[0].message.content

        print(f"Response: {generated_sql}")

        sql_code_blocks = re.findall(r"```sql\s*(.*?)```", generated_sql, re.DOTALL)

        if sql_code_blocks:
            generated_sql = sql_code_blocks[0].strip()

        try:
            df = pd.read_sql_query(generated_sql, conn)
            print(f"Query Results: {df}")
        except Exception as e:
            print("Error running the query:", e)

        if create_sql_script:
            pass
        else:
            result

        return result
    
    def create_sql_script(self, user_prompt, save_sql=False):

        def find_latest_script_number(folder_path):
            folder = Path(folder_path)
            max_number = 0

            # Pattern to match filenames like Script-<number>.sql
            pattern = re.compile(r"Script-(\d+)\.sql", re.IGNORECASE)

            for file in folder.glob("Script-*.sql"):
                match = pattern.match(file.name)
                if match:
                    number = int(match.group(1))
                    if number > max_number:
                        max_number = number

            return max_number

        generated_sql = self.ai_response(user_prompt=user_prompt)

        # Extract just the SQL code between ```sql and ```
        sql_code_blocks = re.findall(r"```sql\s*(.*?)```", generated_sql, re.DOTALL)
        description_blocks = re.split(r"```(?:sql)?[\s\S]*?```", generated_sql)

        if sql_code_blocks:
            generated_sql = sql_code_blocks[0].strip()
            generated_description = description_blocks[1].strip()
        else:
            generated_sql = generated_sql.strip()  # fallback
            generated_description = generated_sql.strip()  # fallback

        query_string = f"""{generated_sql}"""

        Author = "AI Generated SQL Query"

        day = datetime.now().day
        month = datetime.now().month
        year = datetime.now().year
        date_obj = datetime(year, month, day)
        create_date = date_obj.strftime("%d/%m/%Y")

        description = generated_description

        header = f"""-- =============================================
-- Author:		{Author}
-- Create date: {create_date}
-- Description:	{description}
-- =============================================
        """

        folder_path = script_dir / "sql-script"
        latest_number = find_latest_script_number(folder_path)
        # print("Latest script number:", latest_number)

        output_path = script_dir / "sql-script" / f"Script-{latest_number+1}.sql"

        # Make sure the directory exists, create if not
        output_path.parent.mkdir(parents=True, exist_ok=True)

        full_sql = header + "\n" + query_string

        if save_sql:
            # Write to file
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(full_sql)
        else:
            pass

        if save_sql == True:
            return f"\nGenerated SQL:\n, {generated_sql}\n\n Query Results: {df}\n SQL file saved to: {output_path}"
        else:
            return f"\nGenerated SQL:\n, {generated_sql}\n\n Query Results: {df}"

In [39]:
AzureAI().ai_response(user_prompt="Show me the top 10 most spender customers")

C:\Users\Admins\AppData\Local\Temp\ipykernel_18948\1824907698.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_df = pd.read_sql_query(schema_query, conn)


Response: Certainly! Here is the SQL query to show you the top 10 customers who have spent the most:

```sql
SELECT TOP 10
    c.CustomerID,
    c.FirstName,
    c.LastName,
    c.CompanyName,
    SUM(od.LineTotal) AS TotalSpent
FROM dbo.Customer c
JOIN dbo.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
JOIN dbo.SalesOrderDetail od ON soh.SalesOrderID = od.SalesOrderID
GROUP BY c.CustomerID, c.FirstName, c.LastName, c.CompanyName
ORDER BY TotalSpent DESC;
``` 

This query retrieves the top 10 customers along with their total spent amount. The results are ordered by the total amount spent in descending order.
Query Results:    CustomerID    FirstName      LastName                     CompanyName  \
0       29736        Terry     Eminhizer      Action Bicycle Specialists   
1       30050      Krishna  Sunkammurali     Metropolitan Bicycle Supply   
2       29546  Christopher          Beck             Bulk Discount Store   
3       29957        Kevin           Liu       Eastside De

C:\Users\Admins\AppData\Local\Temp\ipykernel_18948\1824907698.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(generated_sql, conn)


'Certainly! Here is the SQL query to show you the top 10 customers who have spent the most:\n\n```sql\nSELECT TOP 10\n    c.CustomerID,\n    c.FirstName,\n    c.LastName,\n    c.CompanyName,\n    SUM(od.LineTotal) AS TotalSpent\nFROM dbo.Customer c\nJOIN dbo.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID\nJOIN dbo.SalesOrderDetail od ON soh.SalesOrderID = od.SalesOrderID\nGROUP BY c.CustomerID, c.FirstName, c.LastName, c.CompanyName\nORDER BY TotalSpent DESC;\n``` \n\nThis query retrieves the top 10 customers along with their total spent amount. The results are ordered by the total amount spent in descending order.'